# 毕业论文`GRAD`
## 处理观测数据

---
*@author: Evan*\
*@date: 2023-11-09*

In [5]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('../../src/')
from namelist import *

In [6]:
path = datadir + "OBS/original_CHEM/"
output_path = datadir + "OBS/CHEM/allTime/"
vars = ['AQI','PM2.5','NO2','O3','O3_8h']

## 根据选定城市导出

In [6]:
cities = ['广州','佛山','东莞','深圳','中山','珠海','江门','肇庆','惠州']
columns = ['date','hour','type']+cities

for var in vars:
    dfs = []
    for foldername in os.listdir(path):
        if foldername.startswith("城市"):
            for filename in os.listdir(os.path.join(path, foldername)):
                df = pd.read_csv(os.path.join(path, foldername, filename))
                df_sel = df[[col for col in df.columns if col in columns]]
                data_group = df_sel.groupby('type').get_group(var)
                dfs.append(data_group)
    dfout = pd.concat(dfs,ignore_index=True)
    dfout['datetime'] = pd.to_datetime(dfout['date'].astype(str)+'T'+dfout['hour'].astype(str).str.zfill(2))
    dfout.drop(['date', 'hour', 'type'], axis=1, inplace=True)
    dfout.set_index('datetime',inplace=True)
    dfout.to_excel(output_path + f'city_{var}.xlsx')
    print(f'Complete {var}')


Complete AQI
Complete PM2.5
Complete NO2
Complete O3
Complete O3_8h


## 根据选定站点导出

### 导出站点列表

In [10]:
df_sites = pd.read_excel(path + '_站点列表/站点列表-2022.02.13起.xlsx')
df_sel = df_sites[df_sites['城市'].isin(cities)]
df_sel

,监测点编码,监测点名称,城市,经度,纬度,对照点
343,1345A,广雅中学,广州,113.2347,23.1423,N
344,1346A,市五中,广州,113.2612,23.105,N
345,1347A,天河职幼,广州,NaN,NaN,N
346,1348A,广东商学院,广州,113.3478,23.0916,N
347,1349A,市八十六中,广州,113.4332,23.1047,N
...,...,...,...,...,...,...
1799,3454A,中山南区,中山,113.3528,22.478,N
1945,3623A,西乡,深圳,113.8953,22.5861,N
1947,3625A,容桂街道办,佛山,113.2544,22.7642,N
1948,3626A,南城元岭,东莞,113.7533,23.0202,N


In [36]:
df_sel[pd.isna(df_sel['纬度'])]

,监测点编码,监测点名称,城市,经度,纬度,对照点
345,1347A,天河职幼,广州,NaN,NaN,N


In [38]:
df_sel.to_excel(output_path + '../sitelocation.xlsx',index=False)

### 根据站点列表导出

In [39]:
df_sites = pd.read_excel(output_path + '../sitelocation.xlsx')
df_sites

,监测点编码,监测点名称,城市,经度,纬度,对照点
0,1345A,广雅中学,广州,113.234700,23.142300,N
1,1346A,市五中,广州,113.261200,23.105000,N
2,1347A,天河职幼,广州,113.317001,23.135799,N
3,1348A,广东商学院,广州,113.347800,23.091600,N
4,1349A,市八十六中,广州,113.433200,23.104700,N
...,...,...,...,...,...,...
80,3454A,中山南区,中山,113.352800,22.478000,N
81,3623A,西乡,深圳,113.895300,22.586100,N
82,3625A,容桂街道办,佛山,113.254400,22.764200,N
83,3626A,南城元岭,东莞,113.753300,23.020200,N


In [42]:
sites = df_sites['监测点编码'].to_list()
columns = ['date', 'hour', 'type',] + sites

for var in vars:
    dfs = []
    for foldername in os.listdir(path):
        if foldername.startswith("站点"):
            for filename in os.listdir(os.path.join(path, foldername)):
                df = pd.read_csv(os.path.join(path, foldername, filename))
                df_sel = df[[col for col in df.columns if col in columns]]
                data_group = df_sel.groupby('type').get_group(var)
                dfs.append(data_group)
    dfout = pd.concat(dfs,ignore_index=True)
    dfout['datetime'] = pd.to_datetime(dfout['date'].astype(str)+'T'+dfout['hour'].astype(str).str.zfill(2))
    dfout.drop(['date', 'hour', 'type'], axis=1, inplace=True)
    dfout.set_index('datetime',inplace=True)
    dfout.to_excel(output_path + f'site_{var}.xlsx')
    print(f'Complete {var}')

Complete AQI
Complete PM2.5
Complete NO2
Complete O3
Complete O3_8h


## 拆分成需要时段的数据分别导出

In [11]:
years  = [2014, 2015, 2016, 2019, 2021, 2022]
months = ['Sep', 'Jul']

for var in vars:
    city_data = pd.read_excel(output_path + f'city_{var}.xlsx', index_col=0)
    site_data = pd.read_excel(output_path + f'site_{var}.xlsx', index_col=0)
    
    for year in years:
        for month in months:
            if month == 'Sep':
                start_date = f'{year}-09-01T00'
                end_date = f'{year}-09-30T23'
            elif month == 'Jul':
                start_date = f'{year}-07-01T00'
                end_date = f'{year}-07-31T23'
            
            city_sel = city_data[start_date:end_date]
            # 补充缺失的时间
            city_sel = city_sel.reindex(pd.date_range(start_date, end_date, freq='H'),fill_value=np.nan)
            # 使用三阶样条函数(分段的多项式函数)插值
            # city_sel = city_sel.interpolate(axis=0, method='spline',order=3)
            # # 样条函数无法对开头和末尾的数据做插值处理，所以需要再做一次填充
            # if city_sel.isnull().values.any():
            #     city_sel = city_sel.fillna(method='ffill')
            #     city_sel = city_sel.fillna(method='bfill')
            city_sel.to_excel(eval(f'obs{month}') + f'city_{var}_{year}.xlsx')
            
            site_sel = site_data[start_date:end_date]
            site_sel = site_sel.reindex(pd.date_range(start_date, end_date, freq='H'),fill_value=np.nan)
            # site_sel = site_sel.interpolate(axis=0, method='spline',order=3)
            # if site_sel.isnull().values.any():
            #     site_sel = site_sel.fillna(method='ffill')
            #     site_sel = site_sel.fillna(method='bfill')
            site_sel.to_excel(eval(f'obs{month}') + f'site_{var}_{year}.xlsx')
    print(f'Completed {var}')

Completed AQI
Completed PM2.5
Completed NO2
Completed O3
Completed O3_8h
